In [47]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import csv
from random import randint
import sys

In [48]:
headers = {"User-Agent":"Ubuntu Chromium/76.0.3809.100"}
           #Mozilla/5.0 (X11; Linux x86_64) 
           #AppleWebKit/537.36 (KHTML, like Gecko) 
           #Chrome/76.0.3809.100 Safari/537.36}
ikea_base = 'https://www.ikea.com'
ikea_series = '/us/en/cat/series-series/'
url = ikea_base + ikea_series

r = requests.get(url, headers = headers)
soup = BeautifulSoup(r.text, 'lxml')

# list of all series
# find all series links
series_list = soup.find_all('li', {'class': 'range-catalog-list__item'})

series_link = []
for req in series_list:
    series_href = req.find_all('a', href=True)
    for sh in series_href:
        series_link.append(sh['href'])

In [49]:
print(len(series_link))

239


In [104]:
# for each family get all products
all_product_link = []
all_product_img = []
for re in series_link[:10]:
    url = re
    r = requests.get(url, headers=headers)
    series_soup = BeautifulSoup(r.text, 'lxml')
    item_list = series_soup.find_all('div', {'class': 'product-compact__spacer'})

    item_links = []
    img_links = []
    for il in item_list:
        item_href = il.find_all('a', href=True)
        for ih in item_href:
            item_links.append(ih['href'])
        img_links.append(il.img['src'])
    
    all_product_link.append(item_links)
    all_product_img.append(img_links)
    # 
    # # breath
    # time.sleep(1)

# unique-ness issues
all_product_link_temp = [list(set(apl)) for apl in all_product_link]
# all_product_link_temp = [val for sublist in all_product_link_temp for val in sublist]
# 
all_product_img_temp = [list(set(apl)) for apl in all_product_img]
# all_product_img_temp = [val for sublist in all_product_img_temp for val in sublist]

In [118]:
print(all_product_link_temp)
print(all_product_img_temp)

[['https://www.ikea.com/us/en/p/allemansraetten-meatballs-frozen-84-meat-content-00346407/', 'https://www.ikea.com/us/en/p/allemansraetten-mashed-potatoes-frozen-30351280/', 'https://www.ikea.com/us/en/p/allemansraetten-mix-for-cream-sauce-70354309/', 'https://www.ikea.com/us/en/p/allemansraetten-vegetable-balls-frozen-100-vegetables-vegan-40346434/'], ['https://www.ikea.com/us/en/p/anilinare-notepad-white-blue-60428004/', 'https://www.ikea.com/us/en/p/anilinare-folder-with-stickers-white-blue-70428013/', 'https://www.ikea.com/us/en/p/anilinare-decorative-box-white-blue-90428210/', 'https://www.ikea.com/us/en/p/anilinare-notebook-white-blue-80428003/', 'https://www.ikea.com/us/en/p/anilinare-binder-clip-70428008/', 'https://www.ikea.com/us/en/p/anilinare-notebook-cork-30428010/', 'https://www.ikea.com/us/en/p/anilinare-decorative-box-set-of-2-white-blue-00428196/'], ['https://www.ikea.com/us/en/p/aepplaroe-gateleg-table-outdoor-brown-stained-brown-50208535/', 'https://www.ikea.com/us/e

In [72]:
all_prod = pd.DataFrame(list(zip(all_product_link, all_product_img)), 
                        columns=['link', 'img_url'])

TypeError: 'NoneType' object is not callable

In [70]:
all_product_link = list(set(all_product_link))
all_product_img = list(set(all_product_img))
print(len(all_product_link))
print(len(all_product_img))
all_product_link[0]

440
440


'https://www.ikea.com/us/en/p/duktig-9-piece-fruit-basket-set-30185747/'

In [64]:
all_product_desc = []
for re in all_product_link:
    url = re + '?page=10'
    r = requests.get(url, headers = headers)
    soup = BeautifulSoup(r.text, 'lxml')
    
    product_html = soup.find_all('meta', {'itemprop': 'description'})
   
    # there is probably only one of these anyway
    product_desc = []
    for pd in product_html:
        product_desc.append(pd['content'])
        
    all_product_desc.append(product_desc)
    
    # breath
    time.sleep(1)

In [22]:
output = pd.DataFrame(list(zip(info_picture, info_corpus)),
                      columns = ['img_url', 'description'])

output.to_csv('/home/peter/Documents/projects/insight/copyprisim/results/example.csv')